In [1]:
# Import necessary libraries
import pandas as pd
import json

# Load the CSV dataset
df = pd.read_csv("semi_strut.csv")

# Display the first few rows of the dataset
df.head()

,Document ID,Content
0,1,"{\n ""title"": ""Introduction to Python"",\n ""..."
1,2,"{\n ""title"": ""Data Analysis with Pandas"",\n ..."
2,3,"{\n ""title"": ""Web Development with Flask"",\n..."
3,4,"{\n ""title"": ""Machine Learning with Scikit-L..."
4,5,"{\n ""title"": ""Data Visualization with Matplo..."


In [10]:
# Tokenization function to extract terms from the JSON-like content
# Remember to exact both 
# 1 .Extract terms from various fields (title, keywords, and content)
# 2. Extract terms from sections' titles and content
# 3. Tokenize the content and create a new column "Terms"
# 4. Implement a preprocessing function that converts terms to lowercase, removes punctuation, and removes common stop words.
    # Create another new column "Terms_preprocessed"
# 5. you can display the DataFrame
# def tokenize_content(content):

In [2]:
def tokenize_content(content):
    # Load JSON content
    content_data = json.loads(content)
    
    # Extract terms from various fields
    terms = []

    # Extract terms from 'title', 'author', and 'keywords' fields
    terms.extend(content_data.get('title', '').split())
    terms.extend(content_data.get('author', '').split())
    terms.extend(content_data.get('keywords', []))

    # Extract terms from sections' titles and content
    sections = content_data.get('sections', [])
    for section in sections:
        terms.extend(content_data.get('title', '').split())
        terms.extend(content_data.get('content', '').split())
    
    return terms


terms = tokenize_content(df['Content'][0])
terms

['Introduction',
 'to',
 'Python',
 'John',
 'Doe',
 'Python',
 'programming',
 'beginner',
 'Introduction',
 'to',
 'Python',
 'Introduction',
 'to',
 'Python']

In [3]:
 # 3 Apply the tokenization function to the 'Content' column and store the results in a new column 'Terms'
df['Terms'] = df['Content'].apply(tokenize_content)

# Display the DataFrame with the extracted terms
print(df[['Document ID', 'Terms']])

   Document ID                                              Terms
0            1  [Introduction, to, Python, John, Doe, Python, ...
1            2  [Data, Analysis, with, Pandas, Jane, Smith, Py...
2            3  [Web, Development, with, Flask, Mike, Johnson,...
3            4  [Machine, Learning, with, Scikit-Learn, Emily,...
4            5  [Data, Visualization, with, Matplotlib, Robert...


In [10]:
def preprocess_terms(terms):
    stop_words = set(stopwords.words('english'))
    translator = str.maketrans('', '', string.punctuation)
    
    preprocessed_terms = []
    for term in terms:
        term = term.lower()
        term = term.translate(translator)
        if term not in stop_words:
            preprocessed_terms.append(term)
    
    return preprocessed_terms

# Apply the preprocessing function to the 'Terms' column and create a new column 'Terms_preprocessed'
df['Terms_preprocessed'] = df['Terms'].apply(preprocess_terms)

# Display the DataFrame
df

NameError: name 'stopwords' is not defined

In [7]:
# Initialize an empty inverted index dictionary
inverted_index = {}
# Build the inverted index

for i, doc in enumerate(df.Terms_preprocessed):
    for term in doc.split():
        if term in inverted_index:
            inverted_index[term].add(i+1)
        else: inverted_index[term] = {i+1}

inverted_index

AttributeError: 'DataFrame' object has no attribute 'Terms_preprocessed'

In [39]:
# posting_list = inverted_index['python']
# posting_list

{0, 1, 2, 3, 4}

In [7]:
# Initialize an empty inverted index dictionary the extracted terms 

In [5]:
# perform boolean operations on postings lists for Boolean search operations
# 1. "Python" OR "Pandas"
def or_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            result.append(posting2[p2])
            p2 += 1
        else:
            result.append(posting1[p1])
            p1 += 1
    while p1 < len(posting1):
        result.append(posting1[p1])
        p1 += 1
    while p2 < len(posting2):
        result.append(posting2[p2])
        p2 += 1
    return result
# 2. "Python" AND "data"
def and_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            p2 += 1
        else:
            p1 += 1
    return result

In [6]:
pl_1 = list(inverted_index['python'])
pl_2 = list(inverted_index['pandas'])
print(or_postings(pl_1, pl_2))
print(and_postings(pl_1, pl_2))


NameError: name 'inverted_index' is not defined